## Google Dance

In [1]:
#Importing all the libraries required for scraping

from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
# using the crome driver for scraping

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
# Url for main page
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

# updating the page by clicking on search button
browser.find_by_name('btnG').first.click()

# parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

# storing all the data from the main class
Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
# search variables Name and reviews within main class using for loop
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Arya Dance Academy'}, {'name': 'Center for Modern Dance Education'}, {'name': 'Wild Child Dance Studio'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'Class Act Performing Arts Studio'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'Grooves Unlimited Dance Studio'}, {'name': 'Harmony Dance Center'}, {'name': 'Dance Palooza Studios'}, {'name': 'Village Dance Academy'}, {'name': 'New Jersey Ballet Company and School'}, {'name': 'Starlight Dance Center'}, {'name': 'Dance Academy of North Jersey'}, {'name': 'Marie Wildey School of Dance'}, {'name': 'Kaye Lynn Dance Studio I'}, {'name': 'DDA Dance Academy'}, {'name': 'Uptown Dance Studio'}, {'name': 'DanceWorks Studios'}, {'name': 'The Kennedy Dancers'}, {'name': 'bboy bgirl lifestyle'}, {'name': 'Dance Theatre of New Jersey'}, {'name': 'Next Step Broadway'}]


In [4]:

Data = []
# Using the geo locator for getting Lat and Long
geolocator = Nominatim()

# getting the link for next pages 

pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

#updating each page link with specific nunmbers to go to next page

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

# parsing pages as the pages updated with above url

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))

# search variables Name, url, adress, phone no, zipcode within main class using for loop

    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
#if the url is empty handling the exception

        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')

# scraping for phone number

        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        
# if the main directory is empty, handling the exception
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
        
# getting the address
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])

# cleaning the address based on number of delimters

            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                    print(orgAddress)
                
# Using geolocator to find the lat and long for each address

            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)

# creating a dataframe for Dance
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbs=lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2&tbm=lcl&ei=6vXlXYHzNqOq5wKi_Y2YCw&start=20&sa=N&rlst=f&ved=0ahUKEwiBkJ7d4pjmAhUj1VkKHaJ-A7MQ8tMDCPoC
2
start=20
start=40
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbs=lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2&tbm=lcl&ei=6vXlXYHzNqOq5wKi_Y2YCw&start=40&sa=N&rlst=f&ved=0ahUKEwiBkJ7d4pjmAhUj1VkKHaJ-A7MQ8tMDCPoC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


20
http://justdancinnj.com/
Just+Dancin',+187+Bloomfield+Ave,+Bloomfield,+NJ+07003
["Just Dancin'", ' 187 Bloomfield Ave', ' Bloomfield', ' NJ 07003']
 187 Bloomfield Ave,  Bloomfield, NJ, 07003
187, Bloomfield Avenue, Bloomfield, Essex County, New Jersey, 07003, United States of America
(40.7811202857143, -74.1905645714286)
http://ellisdancestudios.com/
Ellis+Dance+Studio,+234+Franklin+Ave,+Nutley,+NJ+07110
['Ellis Dance Studio', ' 234 Franklin Ave', ' Nutley', ' NJ 07110']
 234 Franklin Ave,  Nutley, NJ, 07110
234, Franklin Avenue, Nutley, Essex County, New Jersey, 07110, United States of America
(40.815737, -74.161944)
http://www.acacnj.org/
All+Children's+Art+Center+of+NJ,+15+John+St,+Bound+Brook,+NJ+08805
["All Children's Art Center of NJ", ' 15 John St', ' Bound Brook', ' NJ 08805']
 15 John St,  Bound Brook, NJ, 08805
15, John Street, Bound Brook, Somerset County, New Jersey, 08805, United States
(40.5618427634078, -74.5334232548041)
http://www.salsafeveron2.com/
Salsa+Fever+On+

1571, US 46, Parsippany-Troy Hills, Morris County, New Jersey, 07054, United States
(40.8640011299848, -74.4003604836965)
http://www.thewestfieldschoolofdance.com/
Westfield+School+of+Dance,+137+Central+Ave,+Westfield,+NJ+07090
['Westfield School of Dance', ' 137 Central Ave', ' Westfield', ' NJ 07090']
 137 Central Ave,  Westfield, NJ, 07090
137, Central Avenue, Westfield, Union County, New Jersey, 07090, United States
(40.6516015364465, -74.346800894661)
http://artistsinmotionnj.com/
Artists+In+Motion+Dance,+3562+NJ-27+N+Suite+123,+Kendall+Park,+NJ+08824
['Artists In Motion Dance', ' 3562 NJ-27 N Suite 123', ' Kendall Park', ' NJ 08824']
 3562 NJ-27 N Suite 123,  Kendall Park, NJ, 08824
Cant Find the address
http://www.danceexpression.com/
Dance+Expression+Dance+Arts,+3339+NJ-94,+Hamburg,+NJ+07419
['Dance Expression Dance Arts', ' 3339 NJ-94', ' Hamburg', ' NJ 07419']
 3339 NJ-94,  Hamburg, NJ, 07419
3339, NJ 94, Hamburg, Sussex County, New Jersey, 07462, United States
(41.1610100477

Cant Find the address
http://www.shootingstardancenj.com/
Shooting+Star+Dance+Center,+403+Broad+St,+Bloomfield,+NJ+07003
['Shooting Star Dance Center', ' 403 Broad St', ' Bloomfield', ' NJ 07003']
 403 Broad St,  Bloomfield, NJ, 07003
403, Broad Street, Bloomfield, Essex County, New Jersey, 07003, United States of America
(40.8070570098998, -74.192464954488)
Cant Find the url
Cranford+School+of+Music+and+Dance,+777+Walnut+Ave,+Cranford,+NJ+07016
['Cranford School of Music and Dance', ' 777 Walnut Ave', ' Cranford', ' NJ 07016']
 777 Walnut Ave,  Cranford, NJ, 07016
777, Walnut Avenue, Cranford, Union County, New Jersey, 07016, United States of America
(40.6356734615385, -74.3038773461538)
http://www.danceclassnj.com/
Dance+Class,+11+Kelly+Driver+Rd,+Clementon,+NJ+08021
['Dance Class', ' 11 Kelly Driver Rd', ' Clementon', ' NJ 08021']
 11 Kelly Driver Rd,  Clementon, NJ, 08021
Cant Find the address
http://www.tlglivingstonnj.com/
The+Little+Gym+of+Livingston,+4215+Town+Center+Way,+Livin

Cant Find the address
http://www.starstruckdance.net/
Starstruck+Dance+Studio,+977+Valley+Rd,+Stirling,+NJ+07980
['Starstruck Dance Studio', ' 977 Valley Rd', ' Stirling', ' NJ 07980']
 977 Valley Rd,  Stirling, NJ, 07980
977, Valley Road, Long Hill Township, Morris County, New Jersey, 07933, United States
(40.6694231751287, -74.4813891308707)
http://www.jilljustindancealliance.com/
Jill+Justin+Dance+Alliance,+1199+Amboy+Ave,+Edison,+NJ+08837
['Jill Justin Dance Alliance', ' 1199 Amboy Ave', ' Edison', ' NJ 08837']
 1199 Amboy Ave,  Edison, NJ, 08837
Amboy Avenue, Edison, Middlesex County, New Jersey, 08837, United States of America
(40.531819, -74.32817)
https://www.misspatti.com/
Miss+Patti's+School+of+Dance,+85+Godwin+Ave+%23+32,+Midland+Park,+NJ+07432
["Miss Patti's School of Dance", ' 85 Godwin Ave %23 32', ' Midland Park', ' NJ 07432']
 85 Godwin Ave %23 32,  Midland Park, NJ, 07432
Cant Find the address
10
start=100
start=120
https://www.google.com/search?q=kids+dance+classes+ne

Cant Find the address
https://www.kayelynndance.com/
Kaye-Lynn+Dance+Studio,+591+Cranbury+Rd,+East+Brunswick,+NJ+08816
['Kaye-Lynn Dance Studio', ' 591 Cranbury Rd', ' East Brunswick', ' NJ 08816']
 591 Cranbury Rd,  East Brunswick, NJ, 08816
591, Cranbury Road, Gillilandtown, East Brunswick Township, Middlesex County, New Jersey, 08816, United States of America
(40.4088022352941, -74.4209964705882)
http://www.danceexpressionsstudio.net/
Dance+Expressions,+915+County+Rd+517,+Hackettstown,+NJ+07840
['Dance Expressions', ' 915 County Rd 517', ' Hackettstown', ' NJ 07840']
 915 County Rd 517,  Hackettstown, NJ, 07840
Cant Find the address
http://www.dance-xperience.com/
Dance+Xperience+LLC,+3117+NJ-38,+Mt+Laurel+Township,+NJ+08054
['Dance Xperience LLC', ' 3117 NJ-38', ' Mt Laurel Township', ' NJ 08054']
 3117 NJ-38,  Mt Laurel Township, NJ, 08054
NJ 38, Laurel Commercial Centre, Mount Laurel Township, Burlington County, New Jersey, 08054, United States
(39.9658203, -74.9200358)
http://ww

734, Grand Avenue, Ridgefield, Bergen County, New Jersey, 07657, United States
(40.840674, -74.005768)
http://movementxpressnj.net/
Movement+Xpress+Dance+Center,+LLC,+493+Beverly+Rancocas+Rd+2nd+Floor,+Willingboro,+NJ+08046
['Movement Xpress Dance Center', ' LLC', ' 493 Beverly Rancocas Rd 2nd Floor', ' Willingboro', ' NJ 08046']
['', 'NJ', '08046']
 493 Beverly Rancocas Rd 2nd Floor,  Willingboro, NJ, 08046
Cant Find the address
http://www.tlgwaynenj.com/
The+Little+Gym+of+Wayne,+South,+1659+Rt+23,+Wayne,+NJ+07470
['The Little Gym of Wayne', ' South', ' 1659 Rt 23', ' Wayne', ' NJ 07470']
['', 'NJ', '07470']
 1659 Rt 23,  Wayne, NJ, 07470
Route 23, Mountain View, Wayne, Passaic County, New Jersey, 07474, United States
(40.8905894, -74.2495863)
http://www.theconnectiononline.org/
The+Connection,+79+Maple+St,+Summit,+NJ+07901
['The Connection', ' 79 Maple St', ' Summit', ' NJ 07901']
 79 Maple St,  Summit, NJ, 07901
The Connection for Women and Families, 79, Maple Street, Summit, Union 

284, Whitehead Avenue, South River, Middlesex County, New Jersey, 08882, United States of America
(40.4499581212121, -74.3775278484848)
http://www.ajnadance.com/
Ajna+Dance+Company,+18+Bleecker+St,+New+York,+NY+10012
['Ajna Dance Company', ' 18 Bleecker St', ' New York', ' NY 10012']
 18 Bleecker St,  New York, NY, 10012
Sheen Center, 18, Bleecker Street, NoHo, Manhattan Community Board 2, Manhattan, New York County, New York, 10012, United States of America
(40.7254212, -73.9932418)
http://coltsneckdance.com/
Colts+Neck+Dance+%26+Performing,+410+NJ-34,+Colts+Neck,+NJ+07722
['Colts Neck Dance %26 Performing', ' 410 NJ-34', ' Colts Neck', ' NJ 07722']
 410 NJ-34,  Colts Neck, NJ, 07722
JCT Route 34/537, Colts Neck Township, Monmouth County, New Jersey, 07722, United States of America
(40.3029295, -74.1779889)
http://www.bollymovesdance.com/
BollyMoves+Dance+Academy,+26+Fela+Dr,+Parlin,+NJ+08859
['BollyMoves Dance Academy', ' 26 Fela Dr', ' Parlin', ' NJ 08859']
 26 Fela Dr,  Parlin, NJ,

301, Union Street, Northwest Estates, Moorestown Township, Burlington County, New Jersey, 08057, United States
(39.963908, -74.9535)


In [5]:
#df1.to_excel("outputDance.xlsx")

# creating a dataframe for Dance
df1


,name,type,url,phoneNo,address,zipcode,location
0,Just Dancin',Dance Classes,http://justdancinnj.com/,,"187 Bloomfield Ave, Bloomfield, NJ, 07003",07003,"[40.7811202857143, -74.1905645714286]"
1,Ellis Dance Studio,Dance Classes,http://ellisdancestudios.com/,,"234 Franklin Ave, Nutley, NJ, 07110",07110,"[40.815737, -74.161944]"
2,All Children's Art Center of NJ,Dance Classes,http://www.acacnj.org/,,"15 John St, Bound Brook, NJ, 08805",08805,"[40.5618427634078, -74.5334232548041]"
3,Salsa Fever On 2 Dance Academy,Dance Classes,http://www.salsafeveron2.com/,,"83 Franklin St, Jersey City, NJ, 07307",07307,"[40.7420044285714, -74.0483392857143]"
4,Showbiz Kidz Dance Center,Dance Classes,http://showbizdancestudio.com/classes/,,"408 NJ-57, Phillipsburg, NJ, 08865",08865,NaN
...,...,...,...,...,...,...,...
175,All for Dance,Dance Classes,http://www.allfordancestudio.com/,6094264050,"1083 Washington Blvd, Robbinsville Twp, NJ, ...",08691,"[40.2226528022744, -74.6349843499167]"
176,Kidville,Dance Classes,http://www.kidville.com/summit,9,"30 Maple St, Summit, NJ, 07901",07901,"[40.7174857423286, -74.3587436861144]"
177,The Central NJ Ballet Theatre,Dance Classes,https://cnjballet.com/,3,"221 Broad St, Florence Township, NJ, 08518",08518,"[40.1207317753743, -74.8050583763107]"
178,New Jersey Workshop For The Arts,Dance Classes,https://www.njworkshopforthearts.com/,,"152 E Broad St, Westfield, NJ, 07090",07090,"[40.6511676530612, -74.3486901020408]"


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'West Essex Music Together'}, {'name': 'Rockness Music'}, {'name': 'The Institute of Music for Children'}, {'name': 'Little Rockers - Cool Kids Music Classes'}, {'name': 'Musical Munchkins of New Jersey'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Music Together of Bergen County'}, {'name': 'Rockness Music'}, {'name': 'Lindeblad School of Music'}, {'name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'name': 'Little Notes Music School'}, {'name': 'Music Together of Montclair'}, {'name': 'ShreeRaga Carnatic Vocal Music School'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'The Extension Division at Cali School of Music'}, {'name': 'West Essex Music Together'}, {'name': 'Rockness Music'}, {'name': 'A Sound Start'}, {'name': 'Edison School of Music and Dance'}, {'name': 'New Jersey Conservatory'}, {'name': 'Music Together of Summit'}]


In [7]:
Data = []
geolocator = Nominatim()
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                    print(orgAddress)
            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5_blXa6rPKLl5gLgpKLoCQ&start=20&sa=N&ved=0ahUKEwjuuvXV45jmAhWislkKHWCSCJ0Q8tMDCMgC
2
start=20
start=40
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5_blXa6rPKLl5gLgpKLoCQ&start=40&sa=N&ved=0ahUKEwjuuvXV45jmAhWislkKHWCSCJ0Q8tMDCMgC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


20
http://www.newarkschoolofthearts.org/
Newark+School+of+the+Arts,+89+Lincoln+Park,+Newark,+NJ+07102
['Newark School of the Arts', ' 89 Lincoln Park', ' Newark', ' NJ 07102']
 89 Lincoln Park,  Newark, NJ, 07102
89, Lincoln Park, Teachers Village, Newark, Essex County, New Jersey, 07102, United States
(40.7256942631579, -74.1794780526316)
https://denville.b2rmusic.com/
Bach+to+Rock+Denville,+3130+NJ-10+%235,+Denville,+NJ+07834
['Bach to Rock Denville', ' 3130 NJ-10 %235', ' Denville', ' NJ 07834']
 3130 NJ-10 %235,  Denville, NJ, 07834
Cant Find the address
Cant Find the url
Music+Therapy+for+all+Ages,+568+Downer+St,+Westfield,+NJ+07090
['Music Therapy for all Ages', ' 568 Downer St', ' Westfield', ' NJ 07090']
 568 Downer St,  Westfield, NJ, 07090
568, Downer Street, Westfield, Union County, New Jersey, 07090, United States
(40.6450615714286, -74.3567345510204)
http://www.creativekidsnj.com/
Creative+Kids,+551+Park+Ave,+Scotch+Plains,+NJ+07076
['Creative Kids', ' 551 Park Ave', ' Sco

511, East 22nd Street, Paterson, Passaic County, New Jersey, 07514, United States
(40.9228734375, -74.1512538125)
http://www.artkidsacademy.com/
http://www.musictogether.com/
Music+Together+LLC,+174+Nassau+St+%23340,+Princeton,+NJ+08542
['Music Together LLC', ' 174 Nassau St %23340', ' Princeton', ' NJ 08542']
 174 Nassau St %23340,  Princeton, NJ, 08542
Cant Find the address
http://www.originalmusicschool.com/
Original+Music+School,+26+Morris+St,+Morristown,+NJ+07960
['Original Music School', ' 26 Morris St', ' Morristown', ' NJ 07960']
 26 Morris St,  Morristown, NJ, 07960
26, Morris Street, Morristown, Morris County, New Jersey, 07960, United States
(40.797363084721, -74.4788314511426)
http://www.themessyartist.com/
The+Messy+Artist,+356+NJ-10,+East+Hanover,+NJ+07936
['The Messy Artist', ' 356 NJ-10', ' East Hanover', ' NJ 07936']
 356 NJ-10,  East Hanover, NJ, 07936
356, NJ 10, East Hanover, Morris County, New Jersey, 07936, United States
(40.8049548961636, -74.3671942167052)
http:

The Ridgewood Conservatory, 409, Sette Drive, Paramus, Bergen County, New Jersey, 07652, United States
(40.9483118, -74.0695741)
http://www.gardenstreetmusic.com/
Garden+Street+School+of+the+Performing+Arts,+1018+Washington+St+2nd+Floor,+Hoboken,+NJ+07030
['Garden Street School of the Performing Arts', ' 1018 Washington St 2nd Floor', ' Hoboken', ' NJ 07030']
 1018 Washington St 2nd Floor,  Hoboken, NJ, 07030
Cant Find the address
http://www.markmurphysmusic.com/
Mark+Murphy's+Music,+25+Scotland+Rd,+South+Orange,+NJ+07079
["Mark Murphy's Music", ' 25 Scotland Rd', ' South Orange', ' NJ 07079']
 25 Scotland Rd,  South Orange, NJ, 07079
Scotland Road, Academy Heights, South Orange, Essex County, New Jersey, 07050, United States of America
(40.757472, -74.24935)
http://www.creativehandsartstudio.com/
Creative+Hands+Art+Studio+%26+Atelier,+14+Kings+Rd,+Madison,+NJ+07940
['Creative Hands Art Studio %26 Atelier', ' 14 Kings Rd', ' Madison', ' NJ 07940']
 14 Kings Rd,  Madison, NJ, 07940
King

203, 11th Street, Hoboken, Hudson County, New Jersey, 07030, United States
(40.7500905416667, -74.029095375)
http://marlboro.schoolofrock.com/
School+of+Rock+Marlboro,+256+Hwy+79,+Morganville,+NJ+07751
['School of Rock Marlboro', ' 256 Hwy 79', ' Morganville', ' NJ 07751']
 256 Hwy 79,  Morganville, NJ, 07751
Cant Find the address
10
start=100
start=120
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5_blXa6rPKLl5gLgpKLoCQ&start=120&sa=N&ved=0ahUKEwjuuvXV45jmAhWislkKHWCSCJ0Q8tMDCMgC
20
https://www.montclairartmuseum.org/
Montclair+Art+Museum,+3+S+Mountain+Ave,+Montclair,+NJ+07042
['Montclair Art Museum', ' 3 S Mountain Ave', ' Montclair', ' NJ 07042']
 3 S Mountain Ave,  Montclair, NJ, 07042
Montclair Art Museum, 3, South Mountain Avenue, Montclair, Essex County, New Jersey, 07042, United States
(40.81860135, -74.2242380490856)
http://www.njaom.com/contact
New+Jersey+Academy+of+Music,+37+Stonehouse+Rd,+Basking+Ridge,+NJ+07920
['New

S. Broad Street, Hamilton Township, Mercer County, New Jersey, 08620, United States
(40.1909633, -74.7114901)
http://www.njschoolofmusic.com/
New+Jersey+School+of+Music+-+Medford,+10+Union+St,+Medford,+NJ+08055
['New Jersey School of Music - Medford', ' 10 Union St', ' Medford', ' NJ 08055']
 10 Union St,  Medford, NJ, 08055
10, Union Street, Medford Township, Burlington County, New Jersey, 08055, United States
(39.899117264414, -74.8263799430322)
http://www.kalaupasana.com/
Kala+Upasana+Music+and+Dance+Academy,+14+Huntington+Rd,+Edison,+NJ+08820
['Kala Upasana Music and Dance Academy', ' 14 Huntington Rd', ' Edison', ' NJ 08820']
 14 Huntington Rd,  Edison, NJ, 08820
14, Huntington Road, Edison, Middlesex County, New Jersey, 08820, United States
(40.5687170430059, -74.3659949253222)
https://realbraveaudio.com/
Real+Brave,+350+Ramapo+Valley+Rd,+Oakland,+NJ+07436
['Real Brave', ' 350 Ramapo Valley Rd', ' Oakland', ' NJ 07436']
 350 Ramapo Valley Rd,  Oakland, NJ, 07436
Ramapo Valley Roa

Woodbury Glassboro Road, Buckingham Village, Gloucester County, New Jersey, 08080, United States of America
(39.7701419, -75.1349224)
http://www.artist-people.com/
Artist+People+%7C+Sip+and+Paint+Studio+%7C+East+Brunswick,+712+NJ-18,+East+Brunswick,+NJ+08816
['Artist People %7C Sip and Paint Studio %7C East Brunswick', ' 712 NJ-18', ' East Brunswick', ' NJ 08816']
 712 NJ-18,  East Brunswick, NJ, 08816
712, NJ 18, Tanners Corner, East Brunswick Township, Middlesex County, New Jersey, 08816, United States of America
(40.4290995, -74.3839451666667)
http://www.idoltimeacademy.com/#!contact/c1z0x
Idol+Time+Vocal+Academy,+2140+NJ-88,+Brick+Township,+NJ+08724
['Idol Time Vocal Academy', ' 2140 NJ-88', ' Brick Township', ' NJ 08724']
 2140 NJ-88,  Brick Township, NJ, 08724
2140, NJ 88, Point Pleasant, Ocean County, New Jersey, 08724, United States of America
(40.0768027142857, -74.0997482857143)
https://stores.musicarts.com/millburn?source=LGOA5LKB
Music+%26+Arts,+518+Millburn+Ave,+Millburn,+

53, Vervalen Street, Closter, Bergen County, New Jersey, 07624, United States
(40.9704506177326, -73.9582628723433)
http://www.drawingroomllc.com/
Drawing+Room+School-Fine+Arts,+27+Central+Ave,+Midland+Park,+NJ+07432
['Drawing Room School-Fine Arts', ' 27 Central Ave', ' Midland Park', ' NJ 07432']
 27 Central Ave,  Midland Park, NJ, 07432
27, Central Avenue, Midland Park, Bergen County, New Jersey, 07432, United States
(40.9980215333333, -74.1524396666667)
18
start=180
start=200
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5_blXa6rPKLl5gLgpKLoCQ&start=200&sa=N&ved=0ahUKEwjuuvXV45jmAhWislkKHWCSCJ0Q8tMDCMgC
0


In [8]:
#df.to_excel("outputMusic.xlsx")

# Creating a dataframe for Music
df2

,name,type,url,phoneNo,address,zipcode,location
0,Newark School of the Arts,Music Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102",07102,"[40.7256942631579, -74.1794780526316]"
1,Bach to Rock Denville,Music Classes,https://denville.b2rmusic.com/,,"3130 NJ-10 %235, Denville, NJ, 07834",07834,NaN
2,Music Therapy for all Ages,Music Classes,NaN,8662793049,"568 Downer St, Westfield, NJ, 07090",07090,"[40.6450615714286, -74.3567345510204]"
3,Creative Kids,Music Classes,http://www.creativekidsnj.com/,9,"551 Park Ave, Scotch Plains, NJ, 07076",07076,"[40.6481634, -74.3948965]"
4,Jazz House Kids,Music Classes,http://www.jazzhousekids.org/,,"347 Bloomfield Ave, Montclair, NJ, 07042",07042,"[40.8097764, -74.2127012]"
...,...,...,...,...,...,...,...
150,Children's Home Montessori School,Music Classes,http://www.childrenshomemontessorischool.com/,8,"777 Anderson Ave, Cliffside Park, NJ, 07010",07010,"[40.831473, -73.985152]"
151,Dance It Up,Music Classes,http://www.danceitupstudios.com/,6095810808,"1800 NJ-33, Hamilton Square, NJ, 08690",08690,"[40.221684, -74.638169]"
152,Paint Fun Studios,Music Classes,http://paintfunstudio.com/,6,"3231 NJ-27, Franklin Park, NJ, 08823",08823,"[40.4339564751802, -74.5460070547637]"
153,Kidville,Music Classes,http://www.kidville.com/closter,2019831551,"53 Vervalen St, Closter, NJ, 07624",07624,"[40.9704506177326, -73.9582628723433]"


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Life Time Athletic'}, {'name': 'Bamboo Shoots Immersion School'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'The GYM For Kids'}, {'name': 'The Little Gym of Montclair'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}]


In [10]:
Data = []
geolocator = Nominatim()
geolocator = Nominatim()
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                    print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"]# + dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



1
/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=vfflXajJAZGa5gK0042QDA&start=20&sa=N&ved=0ahUKEwjom8C75JjmAhURjVkKHbRpA8IQ8tMDCKAC
2
start=20
start=40
https://www.google.com/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=vfflXajJAZGa5gK0042QDA&start=40&sa=N&ved=0ahUKEwjom8C75JjmAhURjVkKHbRpA8IQ8tMDCKAC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  
C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the defaul

20
http://kidsinsports.com/
['Sports', 'schoolNew', 'York,', 'NY(212)', '744-4900Closed', '⋅', 'Opens', '8:30AM']
Kids+in+Sports+LLC,+1420+2nd+Ave,+New+York,+NY+10021
['Kids in Sports LLC', ' 1420 2nd Ave', ' New York', ' NY 10021']
 1420 2nd Ave,  New York, NY, 10021
1420, 2nd Avenue, Franklin Plaza, Lenox Hill, Manhattan Community Board 8, Manhattan, New York County, New York, 10021, United States of America
(40.770143, -73.9571674)
http://www.ffmanj.com/
['Martial', 'arts', 'schoolJersey', 'City,', 'NJ(201)', '222-8996Closed', '⋅', 'Opens', '12PMTheir', 'website', 'mentions', 'children', 'and', 'classes']
Family+Fitness+Karate+%26+Kickboxing+-+Central+Ave,+419+Central+Ave,+Jersey+City,+NJ+07307
['Family Fitness Karate %26 Kickboxing - Central Ave', ' 419 Central Ave', ' Jersey City', ' NJ 07307']
 419 Central Ave,  Jersey City, NJ, 07307
419, Central Avenue, Jersey City, Hudson County, New Jersey, 07307, United States of America
(40.7490070909091, -74.0474060909091)
http://www.theli

Sheraton Parsippany Hotel, 199, Smith Road, Parsippany-Troy Hills, Morris County, New Jersey, 07054, United States
(40.8616728, -74.4124167547163)
http://nirvanawellnessnj.com/
['Yoga', 'studioMontville,', 'NJ(973)', '495-0334']
Nirvana+Yoga+and+Wellness+Center,+112+Main+Rd,+Montville,+NJ+07045
['Nirvana Yoga and Wellness Center', ' 112 Main Rd', ' Montville', ' NJ 07045']
 112 Main Rd,  Montville, NJ, 07045
112, Main Road, Boonton, Morris County, New Jersey, 07045, United States of America
(40.9155099288288, -74.3789250895736)
http://www.ajwmartialartsacademy.com/
['Martial', 'arts', 'schoolCity', 'of', 'Orange,', 'NJ(973)', '674-5500Closed', '⋅', 'Opens', '11AM']
AJW+Martial+Arts+Academy+%26+Fitness+Center,+356+Main+St,+City+of+Orange,+NJ+07050
['AJW Martial Arts Academy %26 Fitness Center', ' 356 Main St', ' City of Orange', ' NJ 07050']
 356 Main St,  City of Orange, NJ, 07050
356, Main Street, Orange, Essex County, New Jersey, 07050, United States of America
(40.7728364082726, -74

20
http://www.thelittlegym.com/waldwicknj
['Gymnastics', 'centerWaldwick,', 'NJ(201)', '445-4444']
The+Little+Gym+of+Waldwick,+28+Franklin+Turnpike,+Waldwick,+NJ+07463
['The Little Gym of Waldwick', ' 28 Franklin Turnpike', ' Waldwick', ' NJ 07463']
 28 Franklin Turnpike,  Waldwick, NJ, 07463
The Little Gym, 28, Franklin Turnpike, Waldwick, Bergen County, New Jersey, 07463, United States of America
(41.0094741, -74.1197207)
Cant Find the url
['Fitness', 'center', '·', 'Edgewater', 'City', 'PlaceEdgewater,', 'NJ(855)', '579-3287Closed', '⋅', 'Opens', '9AM']
Sweat+NJ+KIDS,+42+City+Pl,+Edgewater,+NJ+07020
['Sweat NJ KIDS', ' 42 City Pl', ' Edgewater', ' NJ 07020']
 42 City Pl,  Edgewater, NJ, 07020
42, City Place, Edgewater, Bergen County, New Jersey, 07020, United States
(40.8075702819484, -73.9897644932853)
http://www.abccnj.com/
['Martial', 'arts', 'schoolBloomfield,', 'NJ(973)', '744-4934Closed', '⋅', 'Opens', '4PM']
Afro+Brazilian+Cultural+Center+of+New+Jersey,+554+Bloomfield+Ave,+Bl

North-South Freeway, Gloucester Township, Camden County, New Jersey, 08012, United States
(39.807337, -75.058629)
http://www.precisionsportsperformance.com/
['GymEast', 'Hanover,', 'NJ(973)', '585-6008Closed', '⋅', 'Opens', '8AM']
Precision+Sports+Performance,+50+Williams+Pkwy+Unit+D,+East+Hanover,+NJ+07936
['Precision Sports Performance', ' 50 Williams Pkwy Unit D', ' East Hanover', ' NJ 07936']
 50 Williams Pkwy Unit D,  East Hanover, NJ, 07936
Cant Find the address
8
start=80
start=100
https://www.google.com/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=vfflXajJAZGa5gK0042QDA&start=100&sa=N&ved=0ahUKEwjom8C75JjmAhURjVkKHbRpA8IQ8tMDCKAC
2
http://www.aikidocenters.com/
['Aikido', 'schoolEast', 'Hanover,', 'NJ(973)', '599-9300']
Aikido+Centers+of+New+Jersey,+55+Eagle+Rock+Ave,+East+Hanover,+NJ+07936
['Aikido Centers of New Jersey', ' 55 Eagle Rock Ave', ' East Hanover', ' NJ 07936']
 55 Eagle Rock Ave,  East Hanover, NJ, 07936
55, Eagle Rock Avenue, E

In [11]:
#df.to_excel("outputSports.xlsx")

# creating a dataframe for sports
df3


,name,type,url,phoneNo,address,zipcode,location
0,Kids in Sports LLC,Sports Classes,http://kidsinsports.com/,830,"1420 2nd Ave, New York, NY, 10021",10021,"[40.770143, -73.9571674]"
1,Family Fitness Karate & Kickboxing - Central Ave,Sports Classes,http://www.ffmanj.com/,,"419 Central Ave, Jersey City, NJ, 07307",07307,"[40.7490070909091, -74.0474060909091]"
2,The Little Gym of Aberdeen,Sports Classes,http://www.thelittlegym.com/aberdeennj,7327720060,"1121 NJ-34 Suite O, Matawan, NJ, 07747",07747,NaN
3,i9 Sports,Sports Classes,https://www.i9sports.com/ProgramDirector/363,9082123200,NaN,NaN,NaN
4,LittleSport,Sports Classes,https://www.littlesport.net/,,"120 E Kings Hwy %23403, Maple Shade Township...",08052,NaN
...,...,...,...,...,...,...,...
57,Blue Life Karate & Kickboxing Centers,Sports Classes,http://www.bluelifekarate.com/,430,"1882 Springfield Ave, Maplewood, NJ, 07040",07040,"[40.720718, -74.270127]"
58,Little Gym Of Washington Township,Sports Classes,http://www.thelittlegym.com/washingtontownshipnj,8562282818,"5501 NJ-42, Blackwood, NJ, 08012",08012,"[39.807337, -75.058629]"
59,Precision Sports Performance,Sports Classes,http://www.precisionsportsperformance.com/,8,"50 Williams Pkwy Unit D, East Hanover, NJ, 0...",07936,NaN
60,Aikido Centers of New Jersey,Sports Classes,http://www.aikidocenters.com/,9735999300,"55 Eagle Rock Ave, East Hanover, NJ, 07936",07936,"[40.831834, -74.340838]"


## Merged all three dataframe into one DF

In [12]:
# merging all 3 dataframe for music, Dannce and Sports

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

# Cleaning the dataframet to drop null values for location

df = df.dropna(axis=0, subset=['location'])
df

,name,type,url,phoneNo,address,zipcode,location
0,Just Dancin',Dance Classes,http://justdancinnj.com/,,"187 Bloomfield Ave, Bloomfield, NJ, 07003",07003,"[40.7811202857143, -74.1905645714286]"
1,Ellis Dance Studio,Dance Classes,http://ellisdancestudios.com/,,"234 Franklin Ave, Nutley, NJ, 07110",07110,"[40.815737, -74.161944]"
2,All Children's Art Center of NJ,Dance Classes,http://www.acacnj.org/,,"15 John St, Bound Brook, NJ, 08805",08805,"[40.5618427634078, -74.5334232548041]"
3,Salsa Fever On 2 Dance Academy,Dance Classes,http://www.salsafeveron2.com/,,"83 Franklin St, Jersey City, NJ, 07307",07307,"[40.7420044285714, -74.0483392857143]"
5,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102",07102,"[40.7256942631579, -74.1794780526316]"
...,...,...,...,...,...,...,...
391,My Gym,Sports Classes,http://www.mygym.com/cherryhill,9,"170 Barclay Farms Shopping Center, Cherry Hi...",08034,"[39.910505, -74.9959876303868]"
392,Blue Life Karate & Kickboxing Centers,Sports Classes,http://www.bluelifekarate.com/,430,"1882 Springfield Ave, Maplewood, NJ, 07040",07040,"[40.720718, -74.270127]"
393,Little Gym Of Washington Township,Sports Classes,http://www.thelittlegym.com/washingtontownshipnj,8562282818,"5501 NJ-42, Blackwood, NJ, 08012",08012,"[39.807337, -75.058629]"
395,Aikido Centers of New Jersey,Sports Classes,http://www.aikidocenters.com/,9735999300,"55 Eagle Rock Ave, East Hanover, NJ, 07936",07936,"[40.831834, -74.340838]"


In [13]:
# exporting to excel

df.to_excel("outputKidsActivities.xlsx") 

In [14]:
# reseting the index

df=df.reset_index()
df.head()

,index,name,type,url,phoneNo,address,zipcode,location
0,0,Just Dancin',Dance Classes,http://justdancinnj.com/,,"187 Bloomfield Ave, Bloomfield, NJ, 07003",07003,"[40.7811202857143, -74.1905645714286]"
1,1,Ellis Dance Studio,Dance Classes,http://ellisdancestudios.com/,,"234 Franklin Ave, Nutley, NJ, 07110",07110,"[40.815737, -74.161944]"
2,2,All Children's Art Center of NJ,Dance Classes,http://www.acacnj.org/,,"15 John St, Bound Brook, NJ, 08805",08805,"[40.5618427634078, -74.5334232548041]"
3,3,Salsa Fever On 2 Dance Academy,Dance Classes,http://www.salsafeveron2.com/,,"83 Franklin St, Jersey City, NJ, 07307",07307,"[40.7420044285714, -74.0483392857143]"
4,5,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102",07102,"[40.7256942631579, -74.1794780526316]"


In [15]:
# renaming the column to _id

df = df.rename(columns={'index':'_id'})
df = df.drop(df.index[0])
#df = df.drop(columns=['Reviews'])
#df = df.drop(columns=['_id'])
df.head()

,_id,name,type,url,phoneNo,address,zipcode,location
1,1,Ellis Dance Studio,Dance Classes,http://ellisdancestudios.com/,,"234 Franklin Ave, Nutley, NJ, 07110",07110,"[40.815737, -74.161944]"
2,2,All Children's Art Center of NJ,Dance Classes,http://www.acacnj.org/,,"15 John St, Bound Brook, NJ, 08805",08805,"[40.5618427634078, -74.5334232548041]"
3,3,Salsa Fever On 2 Dance Academy,Dance Classes,http://www.salsafeveron2.com/,,"83 Franklin St, Jersey City, NJ, 07307",07307,"[40.7420044285714, -74.0483392857143]"
4,5,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102",07102,"[40.7256942631579, -74.1794780526316]"
5,6,Dance Starz Academy of NJ,Dance Classes,http://dancestarznj.net/,,"496 Inman Ave, Rahway, NJ, 07065",07065,"[40.6001789, -74.3113377]"


In [16]:
# adding to directory

split_data = df.to_dict('records')
split_data

[{'_id': 1,
  'name': 'Ellis Dance Studio',
  'type': 'Dance Classes',
  'url': 'http://ellisdancestudios.com/',
  'phoneNo': '',
  'address': ' 234 Franklin Ave,  Nutley, NJ, 07110',
  'zipcode': '07110',
  'location': [40.815737, -74.161944]},
 {'_id': 2,
  'name': "All Children's Art Center of NJ",
  'type': 'Dance Classes',
  'url': 'http://www.acacnj.org/',
  'phoneNo': '',
  'address': ' 15 John St,  Bound Brook, NJ, 08805',
  'zipcode': '08805',
  'location': [40.5618427634078, -74.5334232548041]},
 {'_id': 3,
  'name': 'Salsa Fever On 2 Dance Academy',
  'type': 'Dance Classes',
  'url': 'http://www.salsafeveron2.com/',
  'phoneNo': '',
  'address': ' 83 Franklin St,  Jersey City, NJ, 07307',
  'zipcode': '07307',
  'location': [40.7420044285714, -74.0483392857143]},
 {'_id': 5,
  'name': 'Newark School of the Arts',
  'type': 'Dance Classes',
  'url': 'http://www.newarkschoolofthearts.org/',
  'phoneNo': '',
  'address': ' 89 Lincoln Park,  Newark, NJ, 07102',
  'zipcode': '07

In [17]:
df.dtypes

_id          int64
name        object
type        object
url         object
phoneNo     object
address     object
zipcode     object
location    object
dtype: object

In [20]:
# Set up the MongoDB connection through pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# Create DB

db = myclient["KidsRidedb3"]

# Create collection and insert all the data into the MongoDB

mycol = db["activityData1"]
mycol.insert_many(split_data)